In [1]:
import gensim
from xlrd import open_workbook
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import pandas as pd
from nltk.corpus import stopwords
import pprint
import nltk
import re
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import json
from gensim import similarities
import numpy as np

In [2]:
#book = open_workbook('ignore/eCAPS_COMM_11072019.xlsx')
book = open_workbook('ignore/County_List.xlsx')
county_list = []
#for testing you need 0. real you will need 1
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    #Uncomment these for real use
    #d.pop('COMM_CD')
    #d.pop('COMM_GP')
    #d.pop('COMM_DSCR_UP')
    #d.pop('DSCR_EXT_DV')
    #d.pop('DSCR_EXT')
    #d.pop('COMM_ITM')
    #d.pop('COMM_DET')
    #d.pop('ACT_INACT_FL')
    #d.pop('NAICS_CD')
    county_list.append(d)

print(len(county_list))
for i in county_list:
    print(i)

5
{'COMM_CLS': '225', 'KEYWD': 'COOLERS, DRINKING WATER (WATER FOUNTAINS)', '': ''}
{'COMM_CLS': '295', 'KEYWD': 'ELEVATORS AND ESCALATORS, BUILDING TYPE', '': ''}
{'COMM_CLS': '370', 'KEYWD': 'FOOD PROCESSING AND CANNING EQUIPMENT AND SUPPLIES', '': ''}
{'COMM_CLS': '410', 'KEYWD': 'FURNITURE: HEALTH CARE AND HOSPITAL FACILITY', '': ''}
{'COMM_CLS': '559', 'KEYWD': 'MASS TRANSPORTATION - RAIL VEHICLE PARTS AND ACCESSORIES', '': 'MASS TRANSPORTATION - RAIL VEHICLE PARTS AND ACCESSORIES'}


In [27]:
       
unspec_df = pd.read_excel('ignore/UNSPSC English v220601 project.xlsx','Sheet1', index_col=None, na_values=['NA'])
unspec = unspec_df.drop(['Family Definition','Family','Family Title','Segment Title','Segment Definition','Segment','Version','Key','Synonym','Acronym'],axis = 1)
cols = ['Class Title','Class Definition']
cols2 =['Commodity Title','Commodity Definition']
col3 = ['class','commodity']
unspec["class"] = unspec[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
unspec["commodity"] = unspec[cols2].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
unspec["combine"]= unspec[col3].apply(lambda row: ', '.join(row.values.astype(str)),axis=1)
unspec["combine"]=[row.replace('nan','')for row in unspec["combine"]]
unspec2 = unspec.drop(['Class','Class Title','Class Definition','Commodity Title','Commodity Definition','class','commodity'],axis=1)
unspec2=unspec2.rename(columns={"Commodity":"level_1"})
nums = unspec2['level_1'].tolist()

In [28]:
county_df = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_CLS', index_col=None, na_values=['NA'])
county_df2 = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_ITM', index_col=None, na_values=['NA'])
county_df.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
county_df2.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
countyresult = pd.merge(county_df,county_df2,on='COMM_CLS')
cr = countyresult.drop(['DSCR_EXT_y','DSCR_EXT_DV_x','DSCR_EXT_x','COMM_DSCR_UP_x','COMM_CD_x','COMM_ITM_x','COMM_GP_x','COMM_DET_x','ACT_INACT_FL_x','NAICS_CD_x','COMM_DET_y','COMM_GP_y','COMM_DET_y','ACT_INACT_FL_y','NAICS_CD_y','COMM_DSCR_UP_y','DSCR_EXT_DV_y'],axis=1)
cr2 = cr.rename(columns={"KEYWD_x":"class","KEYWD_y":"commodity","COMM_CD_y":"level_1"})
cr2["commodity"]= [row.replace(' ',',')for row in cr2["commodity"]]
col4=["class","commodity"]
cr2["combine"]= cr2[col4].apply(lambda row: ', '.join(row.values.astype(str)),axis=1)


In [29]:

df1= cr2[['combine','level_1']]
df2 = df1.append(unspec2,ignore_index=True,sort=False)


In [30]:
df2.head(5)

,combine,level_1
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3510
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3515
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3520
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3525
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",3526


In [31]:
df9 = df2.apply(lambda x: ','.join(x.astype(str)),axis = 1)

In [32]:
df9.head()

0    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
1    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
2    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
3    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
4    AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...
dtype: object

In [33]:
df_clean2 = pd.DataFrame({'clean':df9})

In [38]:
df_clean2['clean']=[row.split(' ') for row in df_clean2['clean']]

In [39]:
df_clean2.head()

,clean
0,"[AIRCRAFT, AND, AIRPORT,, EQUIPMENT,, PARTS,, ..."
1,"[AIRCRAFT, AND, AIRPORT,, EQUIPMENT,, PARTS,, ..."
2,"[AIRCRAFT, AND, AIRPORT,, EQUIPMENT,, PARTS,, ..."
3,"[AIRCRAFT, AND, AIRPORT,, EQUIPMENT,, PARTS,, ..."
4,"[AIRCRAFT, AND, AIRPORT,, EQUIPMENT,, PARTS,, ..."


In [35]:
sent23 = [row.split(',') for row in df_clean2['clean']]

In [37]:
print(sent23[10542:10555])

['Fresh cut amaranthusesFresh cut blooms of amaranthuses Fresh cut upright green amaranthusFresh cut bloom of the upright green variety of amaranthus10311804', 'Fresh cut amaranthusesFresh cut blooms of amaranthuses Fresh cut upright red amaranthusFresh cut bloom of the upright red variety of amaranthus10311805', 'Fresh cut amaryllisesFresh cut blooms of amaryllises nan', 'Fresh cut amaryllisesFresh cut blooms of amaryllises Fresh cut naranja amaryllisFresh cut bloom of the naranja variety of amaryllises10311901', 'Fresh cut amaryllisesFresh cut blooms of amaryllises Fresh cut orange nagano amaryllisFresh cut bloom of the orange nagano variety of amaryllises10311902', 'Fresh cut amaryllisesFresh cut blooms of amaryllises Fresh cut pygmee mini amaryllisFresh cut bloom of the pygmee mini variety of amaryllises10311903', 'Fresh cut amaryllisesFresh cut blooms of amaryllises Fresh cut red lion amaryllisFresh cut bloom of the red lion variety of amaryllises10311904', 'Fresh cut amaryllisesF

In [13]:
print(len(sent23))

163115


In [14]:
model2 = gensim.models.word2vec.Word2Vec(sent23,min_count=1,size=300, workers=6, window= 3)

In [15]:
def NumberofDocs(li,nums):
    shortlist=[]
    number=0.0
    for i in li:
        if(i!='nan'):
            point = model2.wv.similarity(str(i),nums)
            if(point>0.8):
                shortlist.append(i) 
    
    return shortlist

result =NumberofDocs(nums,'4045') 
print(len(result))
print(result)
print(df2.loc[df2['level_1'].isin(result)])

45582
[10101506, 10101510, 10101513, 10101519, 10101520, 10101601, 10101606, 10101706, 10101709, 10101710, 10101712, 10101713, 10101714, 10101715, 10101716, 10101717, 10101718, 10101719, 10101723, 10101725, 10101726, 10101727, 10101728, 10101729, 10101731, 10101732, 10101806, 10101809, 10101810, 10101811, 10101901, 10101908, 10101909, 10102003, 10102004, 10102005, 10102006, 10102008, 10102011, 10102017, 10102104, 10102105, 10102106, 10111307, 10121504, 10121506, 10121507, 10121508, 10121703, 10122003, 10131605, 10141501, 10141504, 10141607, 10141610, 10141611, 10141612, 10151523, 10151524, 10151525, 10151536, 10151538, 10151542, 10151543, 10151544, 10151610, 10151613, 10151615, 10151616, 10151707, 10151807, 10151811, 10151812, 10151813, 10151815, 10151818, 10151819, 10151903, 10151909, 10151911, 10151912, 10151913, 10151914, 10151916, 10151919, 10151922, 10151925, 10151926, 10151927, 10152009, 10152016, 10152019, 10152022, 10152024, 10152026, 10152030, 10152031, 10152032, 10152035, 101

In [16]:
result2 =NumberofDocs(nums,'3526') 
print(len(result2))
print(df2.loc[df2['level_1'].isin(result2)])

92
                                                  combine   level_1
16476          Fracturing equipment,, Gel blending units,  20121209
19256   Air transportation support systems and equipme...  25191501
29747   Veterinary products,In this classification, th...  42121603
29752   Veterinary products,In this classification, th...  42121608
29763   Patient clothing,In this classification, this ...  42131501
...                                                   ...       ...
81741       Beads or beading accessories,, Plastic beads,  60122909
82339          Dairying,, Dairy farm in house processing,  70121505
84728              Civil engineering,, Technical drawing,  81101502
113051  Structures of the tracheo-broncheal tree, natu...  85571325
159423  Ear, nose, mouth and throat, part D,This class...  85892378

[92 rows x 2 columns]


In [17]:
model2.wv.similarity('10101707','4045')

0.78284943

In [18]:
model2.wv.similarity('85903723','4045')

0.84805477

In [19]:
model2.wv.most_similar('3105')[:15]

[(' peripheral', 0.9210455417633057),
 ('Musculoskeletal system-nonaxial upper bones ', 0.9208680391311646),
 ('GRAPHIC', 0.9207330346107483),
 (' Canned or jarred organic bluetta blueberries', 0.9205611944198608),
 ('Prosthetic and orthotic device fitting-multiple body systems',
  0.9204990863800049),
 (' (PROGRESSIVE BULBAR PALSY G12.22)', 0.9203726053237915),
 ('Transport buildings and structures', 0.9202191829681396),
 ('Vascular system-veins ', 0.9201950430870056),
 ('Head and neck', 0.9201488494873047),
 (' thus the term IMMUNOSUPRESSANT MACROLIDES.', 0.9200844764709473)]

In [20]:
"""
book = open_workbook('UNSPSC English v220601 project.xlsx')
'''book = open_workbook('Unspec List2b.xlsx')'''
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

print(len(dict_list))
    
doclist =[]
listOfEntry =[]
entrylist = []
df7 = pd.DataFrame()


for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    elif(entry.get("Family")!="" and entry.get("Class")==""):
        if(listOfEntry!=[]):
            #print(listOfEntry)
            doclist += entrylist
            continue
    elif(entry.get("Commodity")==""):
        continue
    else:
        if(entry.get("Class Definition")!=""):
            f = str(sent_tokenize(entry.get("Class Definition").lower())).strip('[]').strip('\'')
            #print(f)
        else:
            f=str([" "]).strip('[]').strip('\'')
        e = str(sent_tokenize(entry.get("Class Title").lower())).strip('[]').strip('\'')
        #print(e)
        g = str(sent_tokenize(entry.get("Commodity Title").lower())).strip('[]').strip('\'')
        #print(g)
        if(entry.get("Commodity Definition")!=""):
            h = str(sent_tokenize(entry.get("Commodity Definition").lower())).strip('[]').strip('\'')
            #print(h)
        else:
            h = str([" "]).strip('[]').strip('\'')
        i=int(entry.get("Commodity"))
        df7 = df7.append({'class':e+f,'commodity':g+h,'level_1': i},ignore_index=True)
        
  
        
doclist.append(listOfEntry) 

print("done")        




model2.wv.vocab
model2.save("W2V-Model")

text = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"
idt =['101'] 
text = text.lower()
newEntry = []
newEn =[]
stop_words = set(stopwords.words('english'))
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]


Wtest = word_tokenize(text)
Wtest = [ j for j in Wtest if not j in ch ]
Wtest = [w for w in Wtest if not w in stop_words]
newEntry.append(Wtest)

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element+" ")
    return result

newEntry[0] = concatenate_list_data(newEntry[0])
newEntry += idt
newEn.append(newEntry)
print(newEn)

new_mod = gensim.models.Word2Vec.load("W2V-Model")
new_mod.build_vocab(newEn,update = True)
new_mod.train(newEn,total_examples=1,epochs = 1)

new_mod.wv.similarity('30121703.0','101')
new_mod.wv.most_similar_cosmul('101')

new_mod.wv.similar_by_vector('101')

"""

'\nbook = open_workbook(\'UNSPSC English v220601 project.xlsx\')\n\'\'\'book = open_workbook(\'Unspec List2b.xlsx\')\'\'\'\n\'\'\'To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to\n16 in order to make the it work.\'\'\'\ndict_list = []\nsheet = book.sheet_by_index(0)\n# read header values into the list\nkeys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]\n\nfor row_index in range(1, sheet.nrows):\n    d = {keys[col_index]: sheet.cell(row_index, col_index).value\n         for col_index in range(sheet.ncols)}\n    dict_list.append(d)\n\nprint(len(dict_list))\n    \ndoclist =[]\nlistOfEntry =[]\nentrylist = []\ndf7 = pd.DataFrame()\n\n\nfor entry in dict_list:\n    if(entry.get("Family")==""):\n        continue\n    elif(entry.get("Family")!="" and entry.get("Class")==""):\n        if(listOfEntry!=[]):\n            #print(listOfEntry)\n            doclist += entrylist\n            continue\n    elif(entry.get("Commodity")==""):\n        